# **Frame the Problem and Look at the Big Picture**  

#### **1. Define the Objective in Business Terms**  

We were hired by a hospital that is having trouble diagnosing:
- Lung benign tissue cancer
- Lung adenocarcinoma cancer 
- Lung squamous cell carcinoma cancer
- Colon adenocarcinoma cancer
- Colon benign tissue cancer

in a timely matter. 

It is our goal to be able to create a multi class classification model to tell the difference between these 5 classes and therefore help give a quick diagnosis.

Being able to classify these different cancers with better accuracy will greatly help the doctors make the proper diagnosis, and also will help the patients by giving 
them a more accurate diagnosis.

#### **2. How Will the Solution Be Used?**  

This model will be used by doctors to help diagnose their patients with better accuracy. This will lead to more patients being given the proper diagnosis.

#### **3. What Are the Current Solutions/Workarounds (If Any)?**  

Currently, the doctors go off of their own diagnosis, which is a problem because a lot of these cancers dont look all too much different from one another and the doctor many not
be able to diagnose it perfectly everytime which could result in serious harm for the patients.


#### **4. How Should You Frame This Problem?**  

This is an image-based supervised multi class classification problem because we are trying to predict what class each image falls into. The model will be trained on the 25000 pictues
that we have in our dataset.

#### **5. How Should Performance Be Measured? Is the Performance Measure Aligned with the Business Objective?**  
- Accuracy. This is alligned with the business objective because our classes are well balanced, and it would just be a simple number of correct predictions / number of total predictions.


#### **6. What Would Be the Minimum Performance Needed to Reach the Business Objective?**  
The minimum performance needed to reach the business objective would be better than what the doctors correct diagnosis rate. For this task, we will be aiming for 90% accuracy.


#### **7. What Are Comparable Problems? Can You Reuse (Personal or Readily Available) Experience or Tools?**  


There are plent of image based classification problems that have happened and are documented online. We also have a comparable problem that we went over in our machine learning class
whihc is thenthe iris one. We also have the section on computer vision in the book that explains a lot about image based classification and how we can run a machine 
learning model on it.  

#### **8. Is Human Expertise Available?**  
For this project, I do not believe we have any other human expertise on this specific problem, however there are many people and resources out there that can help with image based
classification.


#### **9. How Would You Solve the Problem Manually?**  
If solving this problem manually, one could:  
1. **Analyze past trends**: Identify which flights and routes are most prone to delays.  
2. **Review seasonal patterns**: Account for peak travel times, weather conditions, and airport congestion.  
3. **Use airline reports**: Check for maintenance schedules, pilot shortages, or operational disruptions.  
4. **Compare historical and real-time data**: Look for signs of delays such as **late-arriving aircraft, bad weather, or air traffic control restrictions**.  

1. Go over the differences in the images and group the obvious ones together
2. For any non obvious groups, take those images and compare them closely to all other images that we know and do our best to group them.



These insights will guide **feature selection and model training**.  


#### **10. List the Assumptions You (or Others) Have Made So Far. Verify Assumptions if Possible.**  
- The images are all of the same size
- The samples come from an indpendent sample